Autor: Thomas Fetter
 # 1. 📘 What is Machine Vision?


Machine Vision is a subfield of computer science and engineering that enables machines to interpret and process visual information from the world, much like human vision. It is a core technology in areas such as industrial automation, robotics, quality control, autonomous vehicles, and medical imaging.

At its core, Machine Vision combines several disciplines:
- **Computer Vision**: Algorithms and models that allow machines to understand and analyze images or videos.
- **Image Processing**: Techniques for enhancing, filtering, and transforming images to extract meaningful information.
- **Artificial Intelligence (AI)** and **Machine Learning (ML)**: Used to classify, detect, or segment objects within visual data.

Some common tasks in Machine Vision include:
- Image acquisition (via cameras or sensors)
- Preprocessing (e.g., filtering, normalization)
- Feature extraction
- Object detection and recognition
- Classification
- 3D reconstruction
- Visual inspection and quality control

Machine Vision systems typically follow a pipeline like:

$$
\text{Image Acquisition} \rightarrow \text{Preprocessing} \rightarrow \text{Feature Extraction} \rightarrow \text{Decision Making}
$$

Unlike biological vision, Machine Vision is designed for **specific** tasks, often with **high speed**, **repeatability**, and **accuracy** in controlled environments.

🧠 Fun Fact: While the terms *Machine Vision* and *Computer Vision* are often used interchangeably, Machine Vision typically refers to **industrial applications** where the full pipeline from image capture to decision is automated.

## Beispielbilder TODO:

<table>
<tr>
<td><img src="img/farn50.gif" width="200"></td>
<td><img src="bilder/schneeflocke.jpg" width="200"></td>
<td><img src="bilder/küste.jpg" width="200"></td>
</tr>
<tr>
<td>Farnblatt</td>
<td>Schneeflocke</td>
<td>Küstenlinie</td>
</tr>
</table>

---

In [ ]:
import numpy as np 
import cv2  # OpenCV

# Load the Test-Image TODO: wäre das array
input_image = cv2.imread("img/me1.jpg")

In [ ]:
# Display the input image
print("Input image")
# Bild anzeigen

import matplotlib.pyplot as plt

# OpenCV nutzt BGR – für plt.imshow brauchen wir RGB
image_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

# Bild anzeigen
plt.imshow(image_rgb)
plt.axis("off")  # Achsen ausblenden
plt.title("Mein Bild")
plt.show()


In [ ]:
# Create a Gaussian filter kernel 3x3
kernel = 1.0/16.0 * np.array([[1,2,1], [2,4,2], [1,2,1]])
print("Gaussian Kernel\n{}".format(kernel))

#ALLGEMEINER KERNEL mit auto siegma (-1)
# 1D-Gaussian-Kernel (5x1)
gk_1d = cv2.getGaussianKernel(ksize=5, sigma=-1)
# 2D-Gaussian-Kernel durch äußeres Produkt
gk_2d = gk_1d @ gk_1d.T
#print(gk_2d)

In [ ]:
# Apply the filter
output_image = cv2.filter2D(input_image, -1, kernel)

In [ ]:
# Apply the filter again
output_image = cv2.filter2D(output_image, -1, kernel)

In [ ]:
print("Output image")

# OpenCV nutzt BGR – für plt.imshow brauchen wir RGB
image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

# Bild anzeigen
plt.imshow(image_rgb)
plt.axis("off")  # Achsen ausblenden
plt.title("Mein Bild")
plt.show()

## Bildschärfung durch Subtraktion des Gaussian Blur

Ein häufig genutzter Ansatz zur Bildschärfung basiert auf dem Prinzip:

$
\text{Details} = \text{Originalbild} - \text{Weichzeichnung}
$

Dabei wird das geglättete Bild (z. B. durch einen Gaussian-Blur-Filter) vom Originalbild abgezogen. Übrig bleiben die **feinen Strukturen** und **Kanten**, die durch das Glätten unterdrückt wurden.

Das resultierende "Detailbild" hebt also die **hohen Frequenzen** hervor – die feinen Übergänge im Bild. Diese lassen sich zur **Bildschärfung** verwenden oder als Basis für Kantenextraktion nutzen.

Das Verfahren ist einfach, aber sehr wirkungsvoll und liefert visuell anschauliche Ergebnisse.

## Warum braucht man PIL/Pillow für `display()`?

In Jupyter-Notebooks kann man mit der Funktion `IPython.display.display()` **Bilder direkt inline anzeigen** – aber das funktioniert nur korrekt mit bestimmten Objekttypen.

### ✅ Richtig funktioniert es mit:
- `PIL.Image.Image` → wird automatisch hübsch und korrekt dargestellt

### ❌ Nicht funktioniert es mit:
- `NumPy`-Arrays → werden nicht als Bild erkannt
- `OpenCV`-Bilder (`cv2.imread()`) → sind BGR statt RGB und geben beim `display()` nur ein leeres Objekt zurück



In [ ]:
from IPython.display import display
from PIL import Image

# Blurred Image
blurred_image = cv2.GaussianBlur(input_image, (5,5), 0)

# OpenCV BGR → RGB
blurred_rgb = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB)

# In PIL-Image umwandeln
blurred_pil = Image.fromarray(blurred_rgb)

# Anzeigen
print("Blurred Image")
display(blurred_pil)

# Differenzbild (Details extrahieren)
detailed = np.float32(input_image) - np.float32(blurred_image)
detailed = cv2.normalize(detailed, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
detailed_uint8 = np.uint8(detailed)

# Auch hier: BGR → RGB → PIL
detailed_rgb = cv2.cvtColor(detailed_uint8, cv2.COLOR_BGR2RGB)
detailed_pil = Image.fromarray(detailed_rgb)

print("Detailed Image")
display(detailed_pil)

## Bilddetails verstärken durch Addition der Glättung

Ein eher untypischer, aber interessanter Effekt entsteht, wenn man das geglättete Bild **zum Originalbild addiert** mit einem Verstärkungsfaktor:

$
\text{Detailbild} = \text{Original} + \beta \cdot \text{Blur}
$

Das betont helle Bildanteile und verstärkt großflächige Helligkeit. Im Vergleich zur klassischen Detailextraktion durch Subtraktion ergibt sich ein "überzeichneter" Eindruck, nützlich z. B. als visueller Effekt oder zur Betonung von Strukturen.

Hier wurde testweise $\beta = 5$ verwendet.

In [ ]:
# Differenzbild (Details extrahieren)
detailed = np.float32(input_image) + 5*np.float32(blurred_image)
detailed = cv2.normalize(detailed, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
detailed_uint8 = np.uint8(detailed) # da rgb 255

# Auch hier: BGR → RGB → PIL
detailed_rgb = cv2.cvtColor(detailed_uint8, cv2.COLOR_BGR2RGB)
detailed_pil = Image.fromarray(detailed_rgb)

# OpenCV BGR → RGB
#input_rgb = cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB)

print("Original Image")
display(Image.fromarray(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)))

print("Detailed Image")
display(detailed_pil)

# Canny-Edge-Detection

Die Canny-Kantendetektion ist mehrstufig und nutzt intern tatsächlich Gradientenoperatoren wie Sobel, aber sie ist viel mehr als nur Sobel oder Prewitt.
1. **Glättung mit einem Gaussian-Filter**  
   → reduziert Rauschen und verhindert "falsche" Kanten
2. **Gradientenberechnung** (z. B. mit Sobel)
3. **Non-Maximum Suppression**  
   → unterdrückt unscharfe Übergänge
4. **Hysterese-Schwellenwertverfahren**  
   → verbindet echte Kanten, ignoriert schwache
   


### 📘 Sobel-Operator

Der **Sobel-Operator** ist ein Kantendetektor, der den Gradienten eines Bildes durch gewichtete Ableitungen in $x$- und $y$-Richtung berechnet.  
Er kombiniert eine **Ableitung** mit einer **Glättung** (durch gewichtete Mittelung), wodurch er **rauschrobuster** ist als z. B. der Prewitt-Operator.

#### 🔧 Sobel-Kernel:

$$
G_x =
\begin{bmatrix}
-1 & 0 & 1 \\\\
-2 & 0 & 2 \\\\
-1 & 0 & 1
\end{bmatrix},
\quad
G_y =
\begin{bmatrix}
-1 & -2 & -1 \\\\
0 & 0 & 0 \\\\
1 & 2 & 1
\end{bmatrix}
$$

#### 📐 Gradientenstärke:

$$
G = \sqrt{G_x^2 + G_y^2}
$$

---

### 📘 Prewitt-Operator

Der **Prewitt-Operator** ist ein einfacher Kantendetektor, ähnlich wie Sobel, aber **ohne Gewichtung**.  
Er verwendet gleichmäßige Differenzen ohne Mittelwertfilterung und ist daher **weniger rauschresistent**, dafür aber **etwas schneller**.

#### 🔧 Prewitt-Kernel:

$$
G_x =
\begin{bmatrix}
-1 & 0 & 1 \\\\
-1 & 0 & 1 \\\\
-1 & 0 & 1
\end{bmatrix},
\quad
G_y =
\begin{bmatrix}
-1 & -1 & -1 \\\\
0 & 0 & 0 \\\\
1 & 1 & 1
\end{bmatrix}
$$

#### 📐 Gradientenstärke:

$$
G = \sqrt{G_x^2 + G_y^2}
$$

In [ ]:
from ipywidgets import interact, IntSlider
from IPython.display import display


# In Graustufen umwandeln
gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

# Weichzeichnen mit GaussianBlur zur Rauschreduzierung
blurred = cv2.GaussianBlur(gray, (5, 5), 1.4)

# Canny-Kantendetektion
edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

#  Darstellung mit matplotlib
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
plt.title("Original")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(blurred, cmap="gray")
plt.title("Gaussian Blur") 
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(edges, cmap="gray")
plt.title("Canny-Kanten") 
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:


# Graustufenbild vorbereiten
gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 1.4)

# Interaktive Funktion 
def show_canny(low_threshold, high_threshold):
    edges = cv2.Canny(blurred, low_threshold, high_threshold)
    plt.figure(figsize=(10, 5))
    plt.imshow(edges, cmap='gray')
    plt.title(f"Canny Edges\nlow = {low_threshold}, high = {high_threshold}")
    plt.axis("off")
    plt.show()

# Interaktive Slider
interact(show_canny,
         low_threshold=IntSlider(value=50, min=0, max=255, step=1, description='Low Threshold'),
         high_threshold=IntSlider(value=150, min=0, max=255, step=1, description='High Threshold'));

## Sobel-Filter – Kanten in X- und Y-Richtung erkennen

Der **Sobel-Operator** kombiniert eine Ableitung mit einer Glättung und ist ideal zur Kantenerkennung in horizontaler und vertikaler Richtung:

- **Sobel X** erkennt **vertikale Kanten** (also Helligkeitsunterschiede entlang der X-Achse)
- **Sobel Y** erkennt **horizontale Kanten**

Die verwendete Kernelgröße (`ksize=5`) führt zu **glatteren Gradienten** und ist robuster gegenüber Rauschen als der Standard (`ksize=3`).

Durch die getrennte Darstellung von $G_x$ und $G_y$ kann man anschaulich nachvollziehen, wie sich die **Gradientenrichtung** im Bild ergibt.

In [ ]:
# 1. In Graustufen umwandeln
input_image_gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
input_image_gray_f32 = np.float32(input_image_gray)


# 2. Apply sobel filters for X & Y
sobel_x_array = cv2.Sobel(input_image_gray_f32, -1, 1, 0, ksize=5)
sobel_y_array = cv2.Sobel(input_image_gray_f32, -1, 0, 1, ksize=5)


# 3. Umwandeln in uint8 für Anzeige
sobel_x_u8 = cv2.normalize(sobel_x_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
sobel_y_u8 = cv2.normalize(sobel_y_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)


# 4. In PIL-Images konvertieren (Graustufenbild → mode='L')
sobel_x_image = Image.fromarray(sobel_x_u8, mode='L')
sobel_y_image = Image.fromarray(sobel_y_u8, mode='L')


# Anzeigen
print("Sobel X (horizontal)")
display(sobel_x_image)

print("Sobel Y (vertikal)")
display(sobel_y_image)




## Gradientenstärke (Gradient Magnitude)

Die kombinierte Kantenstärke ergibt sich aus den Einzelrichtungen $G_x$ und $G_y$ durch den euklidischen Betrag:

$$
G = \sqrt{G_x^2 + G_y^2}
$$

Dadurch entsteht ein vollständiges Kantenerkennungsbild, ohne Schwellen, wie bei Canny.  
Diese Berechnung ist die Grundlage für viele Kantendetektoren und auch für frühe CNN-Layer.

In [ ]:
# Compute gradient magnitude
grad_magnitude_gray_f32 = np.sqrt(np.square(sobel_x_array) + np.square(sobel_y_array))

# Normalize to 0-255
norm_grad_magnitude_gray_f32 = cv2.normalize(grad_magnitude_gray_f32, None, alpha = 0, beta = 255.0, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

# 3. Umwandeln in uint8 für Anzeige
norm_grad_magnitude_gray_u8 = norm_grad_magnitude_gray_f32.astype(np.uint8)

# 4. In PIL-Images konvertieren (Graustufenbild → mode='L')
norm_grad_magnitude_gray_image = Image.fromarray(norm_grad_magnitude_gray_u8, mode='L')

print("Gradient-Magnitude-Image")

# Display 
display(norm_grad_magnitude_gray_image)

### From Edges to Angles

In [ ]:
# 1. Winkel berechnen (Richtung) in Radiant: -π bis π
angle = np.arctan2(sobel_y_array, sobel_x_array)

# 2. Normierung: -π…π → 0…1 für Hue
angle_norm = (angle + np.pi) / (2 * np.pi)

# 3. Verwendung der vorhandenen Magnitude (berechnet vorher)
magnitude_norm = cv2.normalize(grad_magnitude_gray_f32, None, 0, 1, cv2.NORM_MINMAX)

# 4. HSV-Bild zusammensetzen
hsv = np.zeros((*angle.shape, 3), dtype=np.float32)
hsv[..., 0] = angle_norm          # Hue = Richtung
hsv[..., 1] = 1.0                 # Saturation = voll
hsv[..., 2] = magnitude_norm      # Value = Kantenstärke

# 5. HSV zu RGB konvertieren
rgb = cv2.cvtColor((hsv * 255).astype(np.uint8), cv2.COLOR_HSV2RGB)

# 6. In PIL-Bild umwandeln und anzeigen
gradient_direction_image = Image.fromarray(rgb)
print("Gradient Direction (Hue = Richtung, Value = Stärke)")
display(gradient_direction_image)




## HOG Detector

## Personen erkennen mit HOG + SVM

Die Histogram of Oriented Gradients (HOG) + Support Vector Machine (SVM) Kombination ist ein klassischer Ansatz zur Objekterkennung.

- HOG extrahiert **Kantenorientierungen** und lokale Gradientenverteilungen
- Ein vortrainierter SVM-Klassifikator erkennt damit **Personen**
- Durch **Non-Maximum Suppression** werden doppelte Boxen entfernt

> Diese Methode ist nicht Deep Learning-basiert, aber effizient und gut geeignet für klassische Bildverarbeitungspipelines.

In [ ]:
from imutils.object_detection import non_max_suppression


# 1. Kopie anlegen, um Originalbild nicht zu verändern
image_copy = input_image.copy()

# 2. HOG + vortrainierter SVM-Personendetektor
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# 3. Personen erkennen
(rects, weights) = hog.detectMultiScale(
    image_copy,                 # Eingabebild
    winStride=(4, 4),           # Schrittweite des Sliding Window
    padding=(8, 8),             # Kontext um das Detektionsfenster
    scale=1.05                 # Skalierungsfaktor für Bildpyramide
)

# 4. Bounding Boxes in (x1, y1, x2, y2) umwandeln
rects_np = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])

# 5. Non-Maximum Suppression (doppelte Boxen entfernen)
rects_nms = non_max_suppression(rects_np, probs=None, overlapThresh=0.65)

# 6. Rechtecke einzeichnen
for (x1, y1, x2, y2) in rects_nms:
    cv2.rectangle(image_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)

# 7. RGB-Konvertierung + Anzeige mit display()
image_result_rgb = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)
image_result_pil = Image.fromarray(image_result_rgb)

print("Erkannte Personen mit Bounding Box (HOG + SVM)")
display(image_result_pil)

# Panorama Stitching mit SIFT + RANSAC

## 🔎 SIFT – Scale-Invariant Feature Transform

SIFT ist ein klassischer Algorithmus zur Erkennung und Beschreibung lokaler Bildmerkmale, die robust gegenüber Skalierung, Rotation, Helligkeit und teilweise sogar Perspektivverzerrung sind.

Der Algorithmus funktioniert in mehreren Schritten:

1. **Extrema-Erkennung im Skalenraum:** Bild wird in verschiedenen Maßstäben analysiert (Difference-of-Gaussian).
2. **Keypoint-Lokalisierung:** Auswahl stabiler Punkte basierend auf Kontrast und Kantennähe.
3. **Orientierungszuweisung:** Jedem Keypoint wird eine Hauptorientierung basierend auf Gradienten gegeben.
4. **Descriptors:** Lokales Patch um jeden Keypoint wird in Gradienten-Histogramme zerlegt → ergibt einen 128D-Deskriptor.

➡️ Die Kombination aus Positions-, Orientierungs- und Deskriptordaten erlaubt robustes Matching über verschiedene Bilder hinweg.

In [ ]:
# Load the Test-Image TODO: wäre das array
input_image_aleft_array = cv2.imread("img/stacka_left.jpeg")


# 1. SIFT-Objekt erzeugen (ab OpenCV 4.4 wieder verfügbar)
sift = cv2.SIFT_create()

# 2. Keypoints & Deskriptoren berechnen
keypoints, descriptors = sift.detectAndCompute(input_image_aleft_array, None)

# 3. Bild mit Keypoints anzeigen
img_sift = cv2.drawKeypoints(
    input_image_aleft_array, keypoints, None,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

# 4. Darstellung
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(img_sift, cv2.COLOR_BGR2RGB))
plt.title(f"SIFT Keypoints (Anzahl: {len(keypoints)})")
plt.axis("off")
plt.show()

## Feature Matching mit SIFT-Deskriptoren zwischen zwei Bildern.

### 🧮 SIFT Feature Matching zwischen zwei Bildern

Nachdem in beiden Bildern Keypoints und Deskriptoren extrahiert wurden, können wir sie miteinander vergleichen. Dazu nutzen wir einen Brute-Force Matcher, der für jeden Deskriptor im ersten Bild den **nächstgelegenen Nachbarn** im zweiten Bild sucht.

Wir verwenden den **L2-Abstand** (euklidische Distanz), der für SIFT-Deskriptoren geeignet ist.

Optional wird zusätzlich **Lowe's Ratio-Test** verwendet, um schlechte Matches zu filtern:  
Wenn das Verhältnis zwischen den zwei besten Matches < 0.75 ist, gilt das Match als eindeutig genug.

In [ ]:
# 1. Zweites Bild laden
input_image_aright_array = cv2.imread("img/stacka_right.jpeg")
if input_image_aright_array is None:
    raise FileNotFoundError("img/stacka_right.jpeg wurde nicht gefunden!")

# 2. SIFT-Deskriptoren für beide Bilder berechnen
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(input_image_aleft_array, None)
kp2, des2 = sift.detectAndCompute(input_image_aright_array, None)

# 3. Matcher initialisieren (Brute-Force mit L2)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# 4. K-nächste Nachbarn finden (k=2 für Ratio-Test)
matches = bf.knnMatch(des1, des2, k=2)

# 5. Lowe's Ratio-Test anwenden
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

print(f"Gute Matches: {len(good_matches)}")

# 6. Matches visualisieren
match_vis = cv2.drawMatches(
    input_image_aleft_array, kp1,
    input_image_aright_array, kp2,
    good_matches, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

# 7. Plot anzeigen
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(match_vis, cv2.COLOR_BGR2RGB))
plt.title("SIFT Feature Matches (nach Ratio-Test)")
plt.axis("off")
plt.show()

### 🔁 RANSAC-basierte Homographie-Schätzung

Nicht alle Matches zwischen zwei Bildern sind korrekt, manche sind falsch aufgrund von Rauschen, Wiederholungen oder Bewegung.  
RANSAC (Random Sample Consensus) ist ein robuster Schätzer, der mit solchen Ausreißern umgehen kann.

**Ablauf von RANSAC für Homographie:**

1. Wähle zufällig 4 Punktpaare aus den Matches.
2. Berechne die Homographiematrix $H$ aus diesen 4 Paaren.
3. Wende $H$ auf alle Punkte an und prüfe, wie gut die transformierten Punkte zu den tatsächlichen Punkten passen (Abstand).
4. Zähle, wie viele Paare als "Inlier" gelten (z. B. Abstand < 3 Pixel).
5. Wiederhole viele Male (z. B. 1000 Iterationen).
6. Wähle die $H$ mit den meisten Inliers und berechne daraus die finale Homographie.

➡️ Diese Matrix $H$ kann später verwendet werden, um das zweite Bild in das erste einzustitchen.

In [ ]:
# 1. Extrahiere die korrespondierenden Punkte aus den Matches
pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])

# 2. Homographie mit RANSAC berechnen
H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC, ransacReprojThreshold=4.0)

# 3. Maske gibt an, welche Matches als Inlier gelten
matches_mask = mask.ravel().tolist()

print(f"Gefundene Inliers: {sum(matches_mask)} / {len(matches_mask)}")

# 4. Matches mit Inliers visualisieren
match_vis_inliers = cv2.drawMatches(
    input_image_aleft_array, kp1,
    input_image_aright_array, kp2,
    good_matches, None,
    matchColor=(0, 255, 0),              # grün = inliers
    singlePointColor=(255, 0, 0),
    matchesMask=matches_mask,
    flags=cv2.DrawMatchesFlags_DEFAULT
)

plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(match_vis_inliers, cv2.COLOR_BGR2RGB))
plt.title("RANSAC-Inliers für Homographie")
plt.axis("off")
plt.show()

### 🧵 Bildwarping & Panorama-Stitching

Die berechnete Homographiematrix $H$ beschreibt, wie Punkte aus dem zweiten Bild (`img2`) perspektivisch auf das erste Bild (`img1`) abgebildet werden.

Daher können wir mit `cv2.warpPerspective(...)` das zweite Bild so transformieren, dass es auf das Koordinatensystem des ersten Bildes passt.

Um das Panorama zu erstellen:
1. **Transformiere** das zweite Bild mit $H$ auf die Fläche des ersten.
2. **Erzeuge ein leeres Gesamtbild**, das beide Bilder aufnehmen kann.
3. **Kopiere das erste Bild** direkt ins Panorama.
4. **Lege das transformierte Bild** darüber.

In [ ]:
# Schritt 1: Dimensionen der Bilder
h1, w1 = input_image_aleft_array.shape[:2]
h2, w2 = input_image_aright_array.shape[:2]

# Schritt 2: Ecken des zweiten Bildes transformieren
corners_img2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
transformed_corners = cv2.perspectiveTransform(corners_img2, H)

# Schritt 3: Grenzen des neuen Panoramabildes berechnen
all_corners = np.concatenate((transformed_corners, np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)), axis=0)
[x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
[x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)

translation = [-x_min, -y_min]
width = x_max - x_min
height = y_max - y_min

# Schritt 4: Zweites Bild warpen
H_translation = np.array([[1, 0, translation[0]], [0, 1, translation[1]], [0, 0, 1]])  # für Offset-Korrektur
result = cv2.warpPerspective(input_image_aright_array, H_translation @ H, (width, height))

# Schritt 5: Erstes Bild einfügen
result[translation[1]:translation[1] + h1, translation[0]:translation[0] + w1] = input_image_aleft_array

# Ergebnis anzeigen
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.title("Finales Panorama")
plt.axis("off")
plt.show()

### ✅ Voraussetzungen für gutes Panorama Stitching mit SIFT & RANSAC

Damit zwei Bilder erfolgreich zu einem Panorama zusammengesetzt werden können, sollten sie folgende Eigenschaften aufweisen:

---

#### 1. 🔄 Überlappung

Die Bilder sollten sich in mindestens **30–50 %** ihrer Fläche überlappen.  
SIFT kann nur dort arbeiten, wo **gemeinsame Merkmale** in beiden Bildern sichtbar sind.

📌 _Je mehr Überlappung, desto stabiler die Homographie._

---

#### 2. 💡 Ähnliche Belichtung & Schärfe

Beleuchtung, Kontrast und Bildschärfe sollten **vergleichbar** sein.  
Große Unterschiede können zu fehlerhaften oder fehlenden Matches führen.

---

#### 3. 🧭 Perspektive & Kamerabewegung

Die Kamera sollte sich idealerweise nur leicht **drehen oder schwenken**.  
Starke Translation oder Bewegung auf das Objekt zu bzw. davon weg kann zu fehlerhafter Geometrie führen.

📌 _Am besten geeignet: Rotation um die y-Achse (z. B. mit Stativ)._

---

#### 4. 🔳 Strukturierte Szene

Die Szene sollte **viele visuelle Merkmale** enthalten – wie Ecken, Kanten, Muster, Texturen.  
SIFT braucht Details, um stabile Keypoints zu finden.

⛔ Vermeide:
- glatte Flächen (z. B. Himmel, weiße Wände)
- sich bewegende Objekte (z. B. Menschen, Autos)

---

#### 5. 🚫 Keine starken Verzerrungen

Starke Weitwinkel- oder Fischaugenverzerrung kann die Homographie zerstören.  
Solche Bilder sollten vorher entzerrt werden (z. B. mit `cv2.undistort(...)` und Kamerakalibrierung).

---

### 🎁 Bonus-Tipp:

Wenn du selbst Bilder aufnehmen möchtest:

- Nutze das **Querformat**
- Bewege die Kamera **gleichmäßig mit Überlapp**
- **Vermeide Bewegung in der Szene** (z. B. Personen)
- Halte **Horizont und Bildachse konstant**

➡️ So können saubere, robuste Stitching-Ergebnisse erzielt werden!

# 🧠 Viola-Jones Face Detection

Das Viola-Jones-Verfahren ist ein klassischer Algorithmus zur schnellen und zuverlässigen **Gesichtserkennung in Echtzeit**.  
Es wurde 2001 von Paul Viola und Michael Jones vorgestellt und war der erste Echtzeit-Detektor für Gesichter, der auch auf schwacher Hardware funktioniert.

#### 🚧 Grundprinzipien:

1. **Haar-Like Features:**  
   Merkmale wie Kanten, Kantenverläufe, dunkle/helle Regionen in Rechtecken

2. **Integralbild:**  
   Optimierte Datenstruktur, die Summen in Rechtecken in **konstanter Zeit** berechnet

3. **AdaBoost:**  
   Klassifikator-Kombination – viele schwache Merkmale → starker Entscheidungsbaum

4. **Kaskade aus Klassifikatoren:**  
   Mehrere Klassifikationsstufen prüfen nur weiter, wenn vorherige "ja" sagen  
   → extrem schnell und effizient

#### 📦 Vorteile:

- Echtzeitfähig, auch ohne GPU
- Einfach zu verwenden
- Robust für Frontalgesichter bei guter Beleuchtung

#### ⚠️ Einschränkungen:

- Nicht robust gegenüber Perspektive oder starker Rotation
- Funktioniert schlechter bei Teilverdeckung oder schrägen Gesichtern

In [ ]:
# 1. Graustufenbild erstellen
gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

# 2. Haar-Cascade-Klassifikator laden
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# 3. Gesichter erkennen (angepasste Parameter)
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor=1.03,         # Feiner skalieren
    minNeighbors=2,           # Weniger Nachbarn nötig
    minSize=(20, 20),         # Kleinere Gesichter zulassen
    maxSize=(30, 30),         # Maximalgröße begrenzen
    flags=cv2.CASCADE_SCALE_IMAGE
)

# 4. Bounding Boxes einzeichnen
image_with_faces = input_image.copy()
for (x, y, w, h) in faces:
    cv2.rectangle(image_with_faces, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 5. Anzeige (mit display)
image_faces_rgb = cv2.cvtColor(image_with_faces, cv2.COLOR_BGR2RGB)
image_faces_pil = Image.fromarray(image_faces_rgb)

print(f"Erkannte Gesichter: {len(faces)}")
display(image_faces_pil)

## hint: warning: man muss sehr mit den Parametern spielen!

#### Frontal
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
#### Profil (seitlich)
profile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_profileface.xml")

### Grenzen von Viola-Jones
Funktioniert nicht zuverlässig bei:

-Sonnenbrillen 😎
-starker Rotation / schräge Winkel
-verdeckten Gesichtern
-Liefert keine Gesichtslandmarken (z. B. Augen, Nase, Mund)

### Kein modernes CNN → erkennt nur „klassische Gesichter“

# Klassifikation mit einem Convolutional Neural Network (CNN)

Wir trainieren ein CNN mit **PyTorch** auf dem **MNIST-Datensatz**.

- **MNIST** enthält 70.000 handgeschriebene Ziffern (0–9)
- Die Bilder sind 28×28 Pixel groß und in Graustufen
- Ein CNN lernt hier automatisch, relevante Merkmale aus den Bildern zu extrahieren



In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# === 1. Transformation: Tensor + Normalisierung auf [0, 1] ===
transform = transforms.Compose([
    transforms.ToTensor(),  # konvertiert PIL-Bild zu Tensor [0,1]
])

# === 2. MNIST-Datensätze laden ===
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# === 3. DataLoader erstellen ===
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# === 4. Einzelne Bilder anzeigen mit display() ===
examples = next(iter(train_loader))
images, labels = examples

print("Beispiele aus dem MNIST-Datensatz:")
for i in range(8):
    # Tensor (1, 28, 28) → NumPy-Array (28, 28)
    image_np = images[i][0].numpy() * 255  # zurückskalieren weil ToTensor() auf [0, 1] normalisiert
    image_uint8 = image_np.astype(np.uint8)
    image_pil = Image.fromarray(image_uint8, mode='L')  # das sind 8-Bit-Graustufenbilder
    
    print(f"Label: {labels[i].item()}")
    display(image_pil)

## erstes CNN

Wir definieren ein einfaches Convolutional Neural Network (CNN) für die Klassifikation von handgeschriebenen Ziffern im MNIST-Datensatz.

Das Netzwerk besteht aus zwei Faltungsschichten (mit ReLU und MaxPooling), gefolgt von zwei voll verbundenen (Fully-Connected fc) Schichten:

- **Faltungsschichten (Conv2d)**: lernen lokale Bildmerkmale wie Linien, Ecken oder Übergänge
- **Pooling-Schichten (MaxPool2d)**: reduzieren die Bildgröße und machen das Modell robuster gegenüber Verschiebungen
- **Voll verbundene Schichten (Linear)**: klassifizieren auf Basis der extrahierten Merkmale

## 🔍 Struktur & Parameter im CNN

In der Klasse `SimpleCNN` werden mehrere Layer definiert – jede Schicht besitzt:

- eine **Ausgabe-Dimension** (Shape des Aktivierungs-Tensors nach dieser Schicht)
- eine bestimmte Anzahl an **trainierbaren Parametern** (Gewichte)

Hier ist die Übersicht:

| Layer               | Output Shape         | Parameter Shape           | Anzahl Parameter |
|--------------------|----------------------|---------------------------|------------------|
| Conv2d (1→8, 3x3)  | (8, 28, 28)           | (8, 1, 3, 3)               | 8×1×3×3 = 72     |
| MaxPool2d (2x2)    | (8, 14, 14)           | –                         | 0                |
| Conv2d (8→16, 3x3) | (16, 14, 14)          | (16, 8, 3, 3)              | 16×8×3×3 = 1.152 |
| MaxPool2d (2x2)    | (16, 7, 7)            | –                         | 0                |
| Flatten            | (784,)                | –                         | 0                |
| Linear (784→64)    | (64,)                 | (64, 784) + (64,)          | 50.240           |
| Linear (64→10)     | (10,)                 | (10, 64) + (10,)           | 650              |
| **Gesamt:**        | –                     | –                         | **52.114**       |

> Die Conv- und Linear-Schichten enthalten die trainierbaren Gewichte. 

> Die Shapes der Ausgaben ergeben sich aus dem Bildfluss durch das Netzwerk.  
> Die **Parameter-Formen** ergeben sich aus den Layerdefinitionen – und sind genau die Werte, die durch Backpropagation gelernt werden.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# === Ein einfaches CNN definieren ===
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Eingabe: (1, 28, 28)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)   # → (8, 28, 28)
        self.pool1 = nn.MaxPool2d(2, 2)                                                   # → (8, 14, 14)

        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)                                       # → (16, 14, 14)
        self.pool2 = nn.MaxPool2d(2, 2)                                                   # → (16, 7, 7)

        self.fc1 = nn.Linear(16 * 7 * 7, 64)
        self.fc2 = nn.Linear(64, 10)  # 10 Klassen: Ziffern 0–9

    def forward(self, x):
        x = F.relu(self.conv1(x))   # Convolution + ReLU
        x = self.pool1(x)           # Max-Pooling

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 16 * 7 * 7)  # Flatten
        x = F.relu(self.fc1(x))     # Hidden Layer
        x = self.fc2(x)             # Output Layer
        return x

## Training des CNN

Wir trainieren das Modell mit **CrossEntropyLoss** und dem **Adam-Optimizer**.

- CrossEntropyLoss ist ideal für Klassifikation mit mehreren Klassen
- Adam ist ein adaptiver Optimierer, der schneller konvergiert als SGD

Wir trainieren für 3 Epochen, das genügt bei MNIST meist, um erste Erfolge zu sehen.  
Nach jeder Epoche wird der Loss und die Genauigkeit auf den Trainingsdaten ausgegeben.

## 🎯 CrossEntropyLoss – Klassifikationsfehler bewerten

Die `CrossEntropyLoss` ist die Standard-Loss-Funktion bei **Mehrklassen-Klassifikation** in PyTorch.  
Sie misst, wie "weit entfernt" die Modellvorhersage von der richtigen Klasse ist.

---

### 🔢 Eingaben:

- **`input`**: Rohwerte (Logits) des Modells für jede Klasse  
  → z. B. `[1.2, 0.3, -0.5, 2.7, ...]` für 10 Klassen
- **`target`**: Integer-Label (z. B. `3` für die wahre Klasse „3“)

---

### 🧠 Was passiert intern?

1. **Softmax** konvertiert die Logits in Wahrscheinlichkeiten:

$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$$

2. Dann berechnet Cross Entropy den negativen Log-Wert der Wahrscheinlichkeit der wahren Klasse:

$$
\text{Loss} = -\log(p_{\text{richtig}})
$$

→ je höher die Wahrscheinlichkeit der richtigen Klasse, desto **niedriger der Fehler**.

## 📊 Typische Trainingsstrategien im Deep Learning

Beim Trainieren von neuronalen Netzen – insbesondere Convolutional Neural Networks – ist die richtige Trainingsstrategie entscheidend für gute Generalisierung.

---

### 🔁 Epochen

- Eine **Epoche** bedeutet: Der gesamte Trainingsdatensatz wurde **einmal** durchlaufen.
- Meist werden **mehrere Epochen** benötigt, damit das Modell die Daten richtig lernt.
- Aber: Zu viele Epochen können zum **Overfitting** führen.

---

### 📈 Overfitting vermeiden

Overfitting entsteht, wenn das Modell **zu stark an die Trainingsdaten angepasst ist**, aber **schlecht auf neue, unbekannte Daten** reagiert.

Strategien zur Vermeidung:

- **Early Stopping**: Training wird gestoppt, wenn sich die Testgenauigkeit nicht mehr verbessert
- **Regularisierung** (z. B. `Dropout`, `L2-Weight Decay`)
- **Data Augmentation**: künstliches Vergrößern der Trainingsdaten durch z. B. Rotation, Zoom, Verschiebung

---

### 📉 Lernrate (Learning Rate)

- Die Lernrate (`lr`) bestimmt, wie stark die Gewichte bei jedem Schritt angepasst werden.
- Eine **zu hohe** Lernrate kann das Training instabil machen.
- Eine **zu niedrige** Lernrate kann das Training unnötig langsam machen oder zu schlechten Ergebnissen führen.


In [ ]:
import torch.optim as optim

# === 1. Modell instanziieren ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

# === 2. Loss und Optimizer definieren ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === 3. Trainingsloop ===
n_epochs = 18  # WARNING Too many -> overfitting

for epoch in range(n_epochs):
    running_loss = 0.0                  # Startwert für den Gesamt-Fehler in dieser Epoche, wird unten erhöht
    total, correct = 0, 0               # Zähler für Anzahl aller Bilder (total) und korrekt erkannte Vorhersagen (correct) in dieser Epoche

    model.train()                       # Schaltet das Modell in den Trainingsmodus
    for images, labels in train_loader: # images: Tensor mit Bildern, labels: korrekte Ziffern
        images, labels = images.to(device), labels.to(device)

        # Vorwärtsdurchlauf
        outputs = model(images)         # Führe das Modell mit dem aktuellen Batch aus
        loss = criterion(outputs, labels)   # FBerechne den Fehler zwischen Vorhersage (outputs) und Ziel (labels); criterion ist hier nn.CrossEntropyLoss()

        # Backpropagation
        optimizer.zero_grad()           # Zurücksetzen der Gradienten aus vorherigen Schritten –> sehr wichtig! Sonst würden sich Gradienten aufaddieren
        loss.backward()                 # Berechnet alle Gradienten über Backpropagation
        optimizer.step()                # Anpassung der Gewichte gemäß Gradienten -> der eigentliche "Lernschritt"

        # Statistik sammeln
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)   # Klassenindex mit höchster Wahrscheinlichkeit → das ist die Vorhersage
        total += labels.size(0)                     # Gesamtbilder
        correct += (predicted == labels).sum().item() # Richtige Treffer

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {running_loss:.4f} - Accuracy: {acc:.2f}%")

print("Training abgeschlossen!")

## 🔬 Saliency Map

Eine **Saliency Map** zeigt, welche Pixel im Eingabebild am meisten zur Entscheidung des Modells beitragen.

- Je heller ein Pixel, desto **stärker beeinflusst** es das Ergebnis
- Die Map basiert auf den **Gradienten der Ausgabe** bezogen auf die **Eingabe**
- Verändert man diese Pixel, ändert sich die Vorhersage besonders stark

> So erhält man einen ersten Einblick in das "Wahrnehmungsfeld" eines neuronalen Netzes.

In [ ]:
# Sicherstellen, dass das Modell im Eval-Modus ist
model.eval()

# === Ein einzelnes Bild aus dem Testset nehmen ===
img = images[0].unsqueeze(0).to(device)  # (1, 1, 28, 28)
img.requires_grad_()                     # Aktiviert Gradientenverfolgung für Input

true_label = labels[0].item()

# === Vorhersage und Loss berechnen ===
output = model(img)
loss = criterion(output, torch.tensor([true_label]).to(device))

# === Backward: Gradienten berechnen wrt Input ===
model.zero_grad()
loss.backward()

# === Saliency: Absolutbetrag der Gradienten des Inputs ===
saliency = img.grad.data.abs().squeeze().cpu().numpy()

# === Anzeige ===
plt.figure(figsize=(6, 3))

# Originalbild
plt.subplot(1, 2, 1)
plt.imshow(img.squeeze().detach().cpu().numpy(), cmap='gray')
plt.title(f"Originalbild (Ziffer: {true_label})")
plt.axis("off")

# Saliency Map
plt.subplot(1, 2, 2)
plt.imshow(saliency, cmap='hot')
plt.title("Saliency Map")
plt.axis("off")

plt.tight_layout()
plt.show()

## Test & Vorhersage

Nach dem Training evaluieren wir das Modell auf den Testdaten:

- Die **Testgenauigkeit** zeigt, wie gut das Netzwerk auf unbekannte Bilder generalisiert
- Wir zeigen außerdem einige zufällige Testbilder mit den **Vorhersagen** des Modells

> Dies ist ein klassisches Ende einer Bildklassifikation mit Deep Learning.

In [ ]:
model.eval()  # Inferenzmodus aktivieren (kein Dropout, kein BatchNorm-Update)
correct = 0
total = 0

with torch.no_grad():  # Kein Gradienten-Tracking nötig
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"🔍 Test-Accuracy: {test_accuracy:.2f}%")

In [ ]:
# Eine Mini-Batch aus Testdaten holen
examples = next(iter(test_loader))
images, labels = examples
images, labels = images.to(device), labels.to(device)

# Vorhersagen generieren
with torch.no_grad():
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

print("📸 Zufällige Testbilder mit Vorhersage:")
for i in range(8):
    img = images[i][0].cpu().numpy() * 255
    img_uint8 = img.astype(np.uint8)
    img_pil = Image.fromarray(img_uint8, mode='L')

    print(f"Label: {labels[i].item()} | Vorhersage: {predicted[i].item()}")
    display(img_pil)

## 🏁 Fazit & Ausblick

In diesem Notebook wurde ein vollständiger Einstieg in die Bildverarbeitung und Mustererkennung gegeben – vom klassischen **Sobel-Filter**, über **Kanten- und Merkmalsextraktion**, bis hin zur **modernen Klassifikation mit Convolutional Neural Networks (CNNs)**.

### 🔍 Was haben wir gelernt?

- Wie sich Kanten, Gradienten und Strukturen im Bild erkennen lassen
- Wie klassische Methoden (Gaussian Blur, Sobel, Canny, HOG, Viola-Jones) funktionieren
- Wie ein CNN Bildmerkmale selbstständig lernt und klassifiziert
- Wie man mit `PyTorch` ein eigenes Modell erstellt, trainiert und testet

### 🧠 Was wäre als nächstes spannend?

- **Data Augmentation**: zufällige Rotation, Zoom, Verschiebung → robusteres Modell
- **Dropout / Regularisierung**: Überanpassung verhindern
- **Konfusionsmatrix** und **F1-Score** statt nur Accuracy
- **Transfer Learning**: vortrainiertes Modell (z. B. auf ImageNet) auf eigene Daten anwenden
- **Eigene Datensätze** statt MNIST
- **Explainable AI (XAI)**: z. B. mit Grad-CAM visualisieren, was das Modell „sieht“
